# Programming Assignment 1
## Genre Classification using Locality Sensitive Hashing (LSH)


In [107]:
#Imports

import pandas as pd
import numpy as np
from tqdm import tqdm

### Data Loading and Preprocessing 

In [108]:
"""

"""
# Load data
df_tracks = pd.read_csv('tracks.csv', index_col=0, header=[0, 1])
df_tracks = df_tracks[df_tracks['set']['subset'] == 'medium']
df_features = pd.read_csv('features.csv', index_col=0, header=[0, 1, 2])

# Filter by genres
df_tracks = df_tracks[df_tracks['track']['genre_top'].isin(['Hip-Hop', 'Pop', 'Folk', 'Rock', 'Experimental', 'International', 'Electronic', 'Instrumental'])]

# Split df_tracks into training, testing, and validation sets
df_tracks_train = df_tracks[df_tracks.iloc[:, 30] == 'training']
df_tracks_test = df_tracks[df_tracks.iloc[:, 30] == 'test']
df_tracks_validation = df_tracks[df_tracks.iloc[:, 30] == 'validation']

# Match features with tracks for training, testing, and validation
df_features_train = df_features[df_features.index.isin(df_tracks_train.index)]
df_features_test = df_features[df_features.index.isin(df_tracks_test.index)]
df_features_validation = df_features[df_features.index.isin(df_tracks_validation.index)]

# Extract feature values
X_train = df_features_train.values
X_test = df_features_test.values
X_validation = df_features_validation.values

# Extract genre labels
y_train = df_tracks_train['track']['genre_top']
y_test = df_tracks_test['track']['genre_top']
y_validation = df_tracks_validation['track']['genre_top']

# create smaller test subset from test
X_test_small = X_test[:100]
y_test_small = y_test[:100]

In [109]:
print(y_train)
print(X_train)

track_id
3              Hip-Hop
134            Hip-Hop
136               Rock
139               Folk
198               Folk
              ...     
155297    Instrumental
155298            Folk
155306            Folk
155307    Experimental
155314            Rock
Name: genre_top, Length: 11912, dtype: object
[[ 1.88896334e+00  7.60539293e-01  3.45296562e-01 ...  0.00000000e+00
   1.71672380e+00  6.93301633e-02]
 [ 9.18444753e-01  6.74147248e-01  5.77818275e-01 ...  0.00000000e+00
   1.80610597e+00  5.46228550e-02]
 [ 9.15000617e-01 -6.43476248e-01 -4.60507214e-01 ...  3.41796875e-03
   8.05020452e-01  1.69045236e-02]
 ...
 [ 6.44815028e-01 -8.79404128e-01 -1.14923191e+00 ...  4.88281250e-03
   3.17907929e+00  2.20229235e-02]
 [-4.46937442e-01  1.23500383e+00 -2.50854611e-01 ...  1.95312500e-03
   1.99608481e+00  4.30976301e-02]
 [-1.21936493e-01 -3.48523021e-01 -5.55810153e-01 ...  6.34765625e-03
   1.98907959e+00  1.92883536e-02]]


### Random Projection Matrix

In [110]:
# r_i = rowsize, r_j) = columsize
def generate_random_matrix(r_i, r_j):
    rij = np.random.choice([-1, 0, 1], size=(r_i, r_j), p=[1/6, 2/3, 1/6])
    return np.sqrt(3) * rij

### Hashtable generator function

We use the transpose of the Random Projection Matrix to reduce the dimensionality  and determine the orientation of each track's data relative to the hyperplanes by using the dot Product of the feature matrix and the transposed Random Projection Matrix. 
Then we use the binary representations of the orientations as a bucket and put in the tracks accordingly. 
$ \begin{cases} 
0 & \text{ if } x < 0 \\
1 & \text{ else}
\end{cases}
$ 
We can do this because of $\mathbf{a} \cdot \mathbf{b} = \|\mathbf{a}\| \|\mathbf{b}\| \cos(\theta)$ positive means on one side and negative on the other.
This whole process represents one hashtable.

In [111]:
"""
The binary representations are of length l.
And the number of hashtables we creat is equal to n.
"""
def hashtable_generator(X, l=64, n=2):
    hash_tables_and_matrices = []  
    for _ in range(n):
        buckets = {}
        random_matrix = generate_random_matrix(l, X.shape[1])
        X_dot = np.dot(X, random_matrix.T)
        X_dot = X_dot > 0
        X_dot = X_dot.astype(int)

        for i in range(len(X_dot)):
            hash_str = ''.join(X_dot[i].astype(str))
            if hash_str not in buckets:
                buckets[hash_str] = []
            buckets[hash_str].append(i)
        
        hash_tables_and_matrices.append((buckets, random_matrix))
        
    
    return hash_tables_and_matrices


### Similar Songs Finder
In this step we use the computed hash_tables and the according matrices to find all similar songs of the input song. 
> A music track is defined as similar if it is in the same bucket as $t_i$ in one of the $n$ hash tables.


In [137]:
import itertools

def find_similar_songs(song_input, hash_tables_and_matrices):
    similar_songs_indices = set()

    # First, try to find an exact match in any of the hash table and matrix combinations
    for buckets, random_matrix in hash_tables_and_matrices:
        song_projected = np.dot(song_input, random_matrix.T) > 0
        song_hash = ''.join(song_projected.astype(int).astype(str))

        if song_hash in buckets:
            similar_songs_indices.update(buckets[song_hash])

    # If no exact match is found in any combination, try to find the closest 10 buckets
    if len(similar_songs_indices) == 0:
        print("empty bucket")
        closest_bucket_names = []
        min_hamming_distances = []

        for buckets, random_matrix in hash_tables_and_matrices:
            song_projected = np.dot(song_input, random_matrix.T) > 0
            song_hash = ''.join(song_projected.astype(int).astype(str))

            bucket_hamming_distances = [(hamming_distance(song_hash, bucket_name), bucket_name) for bucket_name in buckets.keys()]
            bucket_hamming_distances.sort(key=lambda x: x[0])
            closest_bucket_names.extend([bucket_name for _, bucket_name in bucket_hamming_distances[:10]])
            min_hamming_distances.extend([hamming_dist for hamming_dist, _ in bucket_hamming_distances[:10]])

        closest_bucket_names = list(dict.fromkeys(closest_bucket_names))  # Remove duplicates
        min_hamming_distances = list(dict.fromkeys(min_hamming_distances))  # Remove duplicates

        for bucket_name, min_hamming_distance in zip(closest_bucket_names, min_hamming_distances):
            for buckets, _ in hash_tables_and_matrices:
                if bucket_name in buckets and buckets[bucket_name]:
                    similar_songs_indices.update(buckets[bucket_name])
                    break

    return list(similar_songs_indices)

def hamming_distance(str1, str2):
    """Calculate the Hamming distance between two binary strings"""
    return sum(c1 != c2 for c1, c2 in zip(str1, str2))

In [113]:
for item in range(len(X_test)):
    test = (find_similar_songs(X_test[0], hashtable_generator(X_train)))
    #print(X_test[0])
    print(y_test.iloc[0])
    #print(X_train[0])
    print(y_train.iloc[0])
    print(y_train.iloc[1])
    print(y_train.iloc[2])
    
    break

Rock
Hip-Hop
Hip-Hop
Rock


### Distance Computation of Similar Songs
This function computes the distance of all similar Songs to the input Song.

In [114]:
def compute_distances(X, song_input, similar_songs, metric="euclid", cut=10):
    filtered_songs = []
    if metric == "euclid":
        for element in similar_songs:
            distance = np.linalg.norm(X[element] - song_input)
            filtered_songs.append((element, distance))
    elif metric == "cosine":
        for element in similar_songs:
            # cosine similarity
            dot_product = np.dot(X[element], song_input)
            norm_song = np.linalg.norm(X[element])
            norm_input = np.linalg.norm(song_input)
            similarity = dot_product / (norm_song * norm_input)
            
            # From similarity to distance (cosine distance)
            distance = 1 - similarity
            filtered_songs.append((element, distance))
    else:
        raise ValueError("Invalid metric specified. Use 'euclid' or 'cosine'.")
    
    sorted_songs = sorted(filtered_songs, key=lambda x: x[1])
    if cut is not None:
        sorted_songs = sorted_songs[:cut]
    
    return [index for index, _ in sorted_songs]

### Getting the Genre by Majority vote

In [115]:
def determine_genre_by_majority_vote(song_indices, Y):
    genres = []
    for index in song_indices:
        genres.append(Y.iloc[index])
    if len(genres) == 0:
        print("No similar songs found.")
        return 'Rock'
    return max(set(genres), key=genres.count)


In [116]:
def find_song_genre(song, X,Y, hashtables = None, l=64, n=2, cut=10, metric="euclid"):
    if hashtables is None:
        hashtables = hashtable_generator(X,l, n)
    similar_songs = find_similar_songs(song, hashtables)
    nearest_neighbours = compute_distances(X,song, similar_songs, metric, cut)
    genre =  determine_genre_by_majority_vote(nearest_neighbours, Y)
    
    return genre 

In [125]:
def test_and_validation_accuracy_with_find_matching_songs_multiple_optimized(l = 64,n = 2,cut = 10, metric="euclid"):
    hashtables = hashtable_generator(X_train, l, n)
    correct = 0
    for index in tqdm(range(len(X_test)), desc="Progress Test Set"):
        song = X_test[index]
        genre = find_song_genre(song, X_train, y_train, hashtables, l, n, cut, metric)
        if genre == y_test.iloc[index]:
            correct += 1
    for index in tqdm(range(len(X_validation)), desc="Progress Validation Set"):
        song = X_validation[index]
        genre = find_song_genre(song, X_train, y_train, hashtables, l, n, cut, metric)
        if genre == y_validation.iloc[index]:
            correct += 1
            
    accuracy = correct / (len(X_test) + len(X_validation))
    
    print("Combined accuracy: ", accuracy)
    return accuracy

test_and_validation_accuracy_with_find_matching_songs_multiple_optimized()

Progress Test Set:  66%|██████▌   | 1006/1535 [00:32<00:13, 39.00it/s]

No similar songs found.


Progress Validation Set: 100%|██████████| 1495/1495 [00:49<00:00, 30.21it/s]

Test accuracy:  0.6155115511551155


0.6155115511551155

In [134]:
def test_accuracy_with_find_matching_songs_multiple_optimized(l = 64,n = 2,cut = 10, metric="euclid"):
    hashtables = hashtable_generator(X_train, l, n)
    correct = 0
    for index in tqdm(range(len(X_test)), desc="Progress"):
        song = X_test[index]
        genre = find_song_genre(song, X_train, y_train, hashtables, l, n, cut, metric)
        if genre == y_test.iloc[index]:
            correct += 1

    accuracy = correct / len(X_test)
    print("Test accuracy: ", accuracy)
    return accuracy

#test_accuracy_with_find_matching_songs_multiple_optimized()

In [139]:
def find_best_parameters_test():
    best_accuracy = 0
    best_parameters = None
    for l in [32, 64, 128]:
        for n in [2, 4, 6, 8]:
            for cut in [5, 10, 15]:
                for metric in ["euclid", "cosine"]:
                    print("Parameters: l=", l, "n=", n, "cut=", cut, "metric=", metric)
                    accuracy = test_accuracy_with_find_matching_songs_multiple_optimized(l, n, cut, metric)
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_parameters = (l, n, cut, metric)
    print("Best parameters: ", best_parameters)
    print("Best accuracy: ", best_accuracy)
    
find_best_parameters_test()

Parameters: l= 128 n= 2 cut= 5 metric= euclid


Progress:   1%|          | 10/1535 [00:00<00:24, 61.48it/s]

empty bucket


Progress:   4%|▍         | 63/1535 [00:00<00:09, 147.46it/s]

empty bucket
empty bucket
empty bucket
empty bucket


...

empty bucket


Progress:  77%|███████▋  | 1189/1535 [00:07<00:02, 145.68it/s]

empty bucket


Progress: 100%|██████████| 1535/1535 [00:09<00:00, 158.16it/s]


empty bucket
empty bucket
empty bucket
Test accuracy:  0.6201954397394137
Parameters: l= 128 n= 4 cut= 5 metric= euclid
